In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

# Load audio file
def load_audio(file_path, sr=16000):
    signal, sr = librosa.load(file_path, sr=sr)
    return signal, sr

# Extract MFCC features
def extract_mfcc(signal, sr, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)
    return mfccs

# Compute fundamental frequency (F0)
def extract_pitch(signal, sr):
    pitches, magnitudes = librosa.piptrack(y=signal, sr=sr)
    pitch = np.max(pitches, axis=0)
    pitch = pitch[pitch > 0]  # Remove zero values
    return np.mean(pitch), np.std(pitch)

# Compute Zero Crossing Rate
def compute_zcr(signal):
    zcr = librosa.feature.zero_crossing_rate(signal)
    return np.mean(zcr), np.std(zcr)

# Compute Spectral Roll-off
def compute_spectral_rolloff(signal, sr):
    rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sr)
    return np.mean(rolloff), np.std(rolloff)

# Compute jitter and shimmer
def compute_jitter_shimmer(signal, sr):
    peaks, _ = find_peaks(signal, height=0)
    intervals = np.diff(peaks) / sr
    jitter = np.std(intervals) / np.mean(intervals) if len(intervals) > 1 else 0
    shimmer = np.std(signal[peaks]) / np.mean(signal[peaks]) if len(peaks) > 1 else 0
    return jitter, shimmer

# Load Pretrained ML Model (Example)
def load_ml_model(model_path):
    model = load_model(model_path)
    return model

# Process Speech Signal
def process_speech(file_path):
    signal, sr = load_audio(file_path)
    mfcc = extract_mfcc(signal, sr)
    pitch_mean, pitch_std = extract_pitch(signal, sr)
    zcr_mean, zcr_std = compute_zcr(signal)
    rolloff_mean, rolloff_std = compute_spectral_rolloff(signal, sr)
    jitter, shimmer = compute_jitter_shimmer(signal, sr)
    
    # Aggregate Features
    features = np.array([pitch_mean, pitch_std, zcr_mean, zcr_std, rolloff_mean, rolloff_std, jitter, shimmer])
    features = features.reshape(1, -1)
    scaler = StandardScaler()
    features = scaler.fit_transform(features)
    
    # Predict Health Metrics (Example)
    # model = load_ml_model("speech_health_model.h5")
    # prediction = model.predict(features)
    
    return {
        "Pitch Mean": pitch_mean,
        "Pitch Std": pitch_std,
        "Zero Crossing Rate Mean": zcr_mean,
        "Zero Crossing Rate Std": zcr_std,
        "Spectral Roll-off Mean": rolloff_mean,
        "Spectral Roll-off Std": rolloff_std,
        "Jitter": jitter,
        "Shimmer": shimmer
    }

# Example Usage
if __name__ == "__main__":
    file_path = "sample_speech.wav"  # Provide speech file path
    results = process_speech(file_path)
    print("Speech Health Metrics:", results)


In [ ]:
pip install librosa numpy matplotlib scipy scikit-learn tensorflow
